In [1]:
# -*- coding:utf-8 -*-
import os
import re
import json
import codecs
import pandas as pd
import data_io as di
from scattertext import chinese_nlp
from scattertext import CorpusFromParsedDocuments
from scattertext import produce_scattertext_explorer
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:98% !important;}</style>"))

由于相比于ipython，notebook在处理大量数据时容易假死，这里为了方便演示，我在代码中设置了如果有处理好的数据文件就直接读入。
因此，**强烈建议**在运行之前先`ipython comment_score_analyse.py`，等其执行完后再继续运行notebook以达到较好的演示效果。

In [2]:
raw_comment_path = './data/raw_comment.json'
movie_info_path = './data/movie_info.csv'
comments_dataframe_path = './data/comment_dataframe.csv'
comments_after_clean = './data/fine_comment.csv'
html_file = './演示/影片质量-评论分析.html'

读取电影基本数据，将其按照豆瓣评分分类，大于7.5分的归类为**好片**，低于5分的归为**烂片**。

In [3]:
movie_info = pd.read_csv(movie_info_path)
good = set()
bad = set()
for i, r in movie_info.iterrows():
    if r['score.douban'] >= 7.5:
        good.add(r['name'])
    if r['score.douban'] <= 5:
        bad.add(r['name'])
print(good, bad)

{'绣春刀II：修罗战场', '建军大业', '战狼2'} {'京城81号Ⅱ', '三生三世十里桃花', '逆时营救', '大闹天竺'}


下面我们将原始评论数据读入，筛出对上面好片和烂片的评论并提取它们的作者，文本内容和情感分数信息。
缺失的评论数据用‘’补全，
处理完成的数据存到本地。

In [4]:
if not os.path.isfile(comments_dataframe_path):
    comments = di.read_multi_json(raw_comment_path)
    comments_dataframe = pd.DataFrame(columns=['Cate', 'Auther', 'Text','Score'])
    for i in range(len(comments)):
        m_name = comments[i]['movieName']
        if m_name in good:
            cate = 'good'
        elif m_name in bad:
            cate = 'bad'
        else:
            continue
        comments_dataframe.loc[i] = [cate, comments[i]['user'].get('displayName', ''), comments[i].get('content', ''), comments[i]['score'] ]
    comments_dataframe['text'].fillna('', inplace=True)
    comments_dataframe.to_csv('./data/comments_dataframe.csv', index=False)
else:
    comments_dataframe = pd.read_csv(comments_dataframe_path)
comments_dataframe.head()

,Cate,Author,Text,Score
0,bad,离歌小三儿,#电影三生三世十里桃花# 2017年暑假上映，@杨洋icon 饰演夜华。高颜值组合共同演绎，...,10
1,bad,司空纭淼,#电影三生三世十里桃花# 最近正好有机会看了电影三生的预告片和定妆照，真心有点意犹未尽。...,10
2,bad,闲人野志,#电影三生三世十里桃花# 三年剧本筹备期，全国各地名胜取景，奥斯卡特效班底，高定服装设计，选...,10
3,bad,杨洋icon48天团经纪人杨阿毛,#杨洋##杨洋icon##杨洋三生三世十里桃花##杨洋夜华# [可爱][可爱][可爱]中国仙...,10
4,bad,微微一笑很倾城肖奈,电影三生三世十里桃花，杨洋饰演太子夜华，期待今年暑假上映。#杨洋三生三世十里桃花# ​,10


去除停用词，除了常见停用词外，这里我们把片名/演员名/角色名的分词结果也当做停用词。

In [5]:
def remove_stop_words(text):
    text = str(text)
    stop_words = '的 了 的 吗 啊 # [ ] 。 ， 2017 三生 三世 十里 桃花 战狼 绣春刀 修罗 战场 京城 81 逆时 营救 建军 大业 大闹 天竺 沈炼 裴纶 叶挺 达康 达康书记 崇祯 王宝强 张翰 大幂幂 书记 白客 建华 杨幂 吴京 弗兰克·格里罗 吴刚 成龙 李治廷 张艺兴 吴亦凡 林更新 姚晨 白客 岳云鹏 彭于晏 倪妮 余文乐 尚雯婕 鲍春来 孙建弘 刘亦菲 杨洋 罗晋 刘德华 姜武 宋佳 刘烨 朱亚文 黄志忠 王景春 欧豪 黄渤 段奕宏 徐静蕾 张震 张译 王凯 张鲁一 林心如 张智霖 梅婷 钟欣潼 金城武 周冬雨 孙艺洲 霍建华 金士杰 余文乐 杨千嬅 蒋梦婕 夜华 冷锋 魏忠贤 信王'.split(' ')
    stop_words = re.compile(r'|'.join(stop_words))
    text = stop_words.sub('', text)
    return text
if not os.path.isfile(comments_after_clean):
    comments_dataframe['Text'] = comments_dataframe.apply(lambda row: remove_stop_words(row['Text']), axis=1)
    comments_dataframe['Text'] = comments_dataframe['Text'].apply(chinese_nlp)
else:
    comments_dataframe = pd.read_csv(comments_after_clean)
comments_dataframe.head()

,Cate,Author,Text,Score
0,bad,离歌小三儿,电影 2017 年 暑假 上映 ， @ 杨洋 icon 饰演...,10
1,bad,司空纭淼,电影 最近 正好 有 机会 看 电影 预告片...,10
2,bad,闲人野志,电影 三年 剧本 筹备 期 ， 全国 各地 名胜 取景 ， 奥斯...,10
3,bad,杨洋icon48天团经纪人杨阿毛,杨洋 杨洋 icon 杨洋 杨洋 夜华 ...,10
4,bad,微微一笑很倾城肖奈,电影 ， 杨洋 饰演 太子 夜华 ， 期待 今年 暑假 上映 杨洋 ...,10


下面我们将分布图画出来，横轴为好片的频率，纵轴为烂片的词频，对于颜色来说，颜色越偏蓝，说明该词的“好片”属性更强，颜色越偏红，则“烂片”属性更强。  
点击其中的数据点可以看到底下会显示详细信息，或者可以在下面的搜索框里搜你想找的词汇。   
例如：我们看到右上角附件有“颜值”一词，在右上角说明其总词频较高，其颜色深红，说明该词往往和烂片伴生。   
如因环境等原因无法查看下图，也可打开*“演示”*文件夹中的*影片质量-评论分析.html*查看，由于数据量较大，加载可能需要较长时间。

In [6]:
if not os.path.isfile(html_file):
    courpus = CorpusFromParsedDocuments(comments_dataframe, category_col='Cate', parsed_col='Text').build()
    html = produce_scattertext_explorer(corpus,
                                        category='good',
                                        category_name='好片',
                                        not_category_name='烂片',
                                        width_in_pixels=1000,
                                        jitter=0.1,
                                        minimum_term_frquency=20,
                                        metadata=comments_dataframe['Author'],
                                        asian_mode=True)

    codecs.open(html_file, 'wb').write(html.encode('utf-8'))
IFrame(src=html_file, width=1200, height=700)